### Step1: LIbrary 불러오기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import torch.nn.init

### Step2: GPU 설정하기

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

### Step3: 학습 Parameter 작성하기 

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size =100

### Step4: Dataset 가져오기 & DataLoader 만들기

In [4]:
mnist_train = datasets.MNIST(root='\MNIST_data/',
                            train=True,
                            transform=transforms.ToTensor(), # 그냥 받아왔을 때에는 Tensor Value가 아니므로 
                            download=True) 

mnist_test = datasets.MNIST(root='\MNIST_data/',
                           train=False,
                           transform=transforms.ToTensor(),
                           download=True)

In [5]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                         batch_size = batch_size, 
                                         shuffle = True, 
                                         drop_last = True)

### Step5: 모델 만들기

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        ##### 추가 #####
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        ##### Fully Connected Layer 하나 더 추가 ####
        self.fc1 = nn.Linear(3*3*128, 625, bias=True)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True) # 10: 0~9
        torch.nn.init.xavier_uniform_(self.fc1.weight) # 요 Layer만 일단 초기화
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1) # Batchsize만큼 펼쳐주고 나머지는 알아서 하도록~
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out

In [7]:
model = CNN().to(device) # cuda 사용 가능하면 cuda, 안되면 cpu
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

##### 사실, 모델을 만들면 바로 모델을 갖다 쓰는 것이 아니라, 임의의 value 값을 넣어서 테스트를 한 번 해보아야.

In [8]:
value = torch.Tensor(1, 1, 28, 28).to(device)
print(model(value).shape)

torch.Size([1, 10])


### Step6: Loss function과 Optimizer 설정하기

In [9]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Step7: 학습 및 Loss check

#### CPU 너무 느려서 학습 포기...

하... 학습도 느리고, 메모리도 없고...

In [10]:
total_batch = len(data_loader)

print('Learning is started. It might take some time.')

for epoch in range(training_epochs):
    avg_cost = 0 # 여기에다가 loss를 하나씩 담을 것 
    
    for X, Y in data_loader:
        # X: image, Y: label
        X = X.to(device) # cuda 연산을 진행하려면 torch.Tensor가 아니라 torch.cuda의 텐서가 되어있어야하기에.
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch 
   
    # 한 Epoch의 학습이 끝나면 Cost 출력
    print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))
    
print('Learning Finished!')


Learning is started. It might take some time.
[Epoch: 1] cost = 0.16173690557479858
[Epoch: 2] cost = 0.04261275753378868
[Epoch: 3] cost = 0.029503794386982918
[Epoch: 4] cost = 0.023244544863700867
[Epoch: 5] cost = 0.018358932808041573
[Epoch: 6] cost = 0.015689043328166008
[Epoch: 7] cost = 0.012389730662107468
[Epoch: 8] cost = 0.009893049485981464
[Epoch: 9] cost = 0.010651451535522938
[Epoch: 10] cost = 0.009349045343697071
[Epoch: 11] cost = 0.008002594113349915


KeyboardInterrupt: 

### Loss Check

In [11]:
with torch.no_grad(): # 핛습을 안 할 거니까!
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)  # 불러와서 한 번에 집어 넣으려고 쭉 펼침
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_preddiction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

RuntimeError: $ Torch: not enough memory: you tried to allocate 0GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:204